In [1]:
from team_code.generate import setup_model_and_tokenizer, generate_text, get_ppl
from tqdm import tqdm
import nltk
nltk.download('punkt')
import telebot
from telebot import types
import math
import os
import uuid
import config

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
model, tokenizer = setup_model_and_tokenizer()

/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.0.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/

In [8]:
bot = telebot.TeleBot(config.token)

In [4]:
def make_dir(dir_name):
    isExist = os.path.exists(dir_name)
    if not isExist:
        os.mkdir(dir_name)

In [5]:
make_dir('audio')
make_dir('ready')
make_dir('voice')
make_dir('photo')

In [9]:
# cur_query_list = []
history_list = ("", "")


@bot.message_handler(commands=['start'])
def send_welcome(message):

    global history_list
    history_list = ("", "")
#     hideBoard = types.ReplyKeyboardRemove()
    msg = bot.send_message(message.chat.id, text ='Здравствуй, {0.first_name}! Я мультимодальный диалоговый ассистент NSU AI, чем могу помочь?'.format(message.from_user))


        
@bot.message_handler(content_types=['text'])
def handle_text(message):
    markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
    global history_list

    try:
        
        if message.text =="Контекст очищен" or message.text == "Generating answer...":
            return
        if message.text == 'Очистить контекст':
            history_list = ("","")
            bot.send_message(message.chat.id, text="Контекст очищен")
        else:
            cur_query_list = []

            msg = bot.send_message(message.chat.id, text="Generating answer...")
            bot.last_message_sent = msg.chat.id, msg.message_id

            cur_query_list.append({'type': 'text', 'content': message.text})
            btn1 = types.KeyboardButton("Очистить контекст")

            markup.add(btn1)
            answer, new_history_list = generate_text(model, tokenizer, cur_query_list=cur_query_list, history_list=history_list)
            history_list = new_history_list
            bot.delete_message(*bot.last_message_sent)
            bot.send_message(message.chat.id, text=answer, reply_markup=markup)
        
        


    except Exception:
        bot.send_message(message.chat.id, text="Попробуйте написать текст запроса снова", reply_markup=markup)


    
@bot.message_handler(content_types=['photo'])
def handle_image(message):
    markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
            
    btn1 = types.KeyboardButton("Очистить контекст")

    markup.add(btn1)
    cur_query_list = []


    try:
        msg = bot.send_message(message.chat.id, text="Generating answer...")
        if message.caption is not None:
            cur_query_list.append({'type': 'text', 'content': message.caption})
        else:
            cur_query_list.append({'type': 'text', 'content': 'Describe what you see in this picture?'})
        bot.last_message_sent = msg.chat.id, msg.message_id
        filename = str(uuid.uuid4())
        file_name_full="./photo/"+filename +'.jpg'
        fileID = message.photo[-1].file_id
        file_info = bot.get_file(fileID)
        photo = bot.download_file(file_info.file_path)
        
        with open(file_name_full, 'wb') as new_file:
            new_file.write(photo)
        cur_query_list.append({'type': 'image', 'content': file_name_full})

        global history_list
        answer, new_history_list = generate_text(model, tokenizer, cur_query_list=cur_query_list, history_list=history_list)
        history_list = new_history_list
        bot.delete_message(*bot.last_message_sent)

        bot.send_message(message.chat.id, text=answer, reply_markup=markup)

       
        
    except Exception:
        
        bot.send_message(message.chat.id, text="Возникла проблема с вашим фото, попробуйте ещё раз", reply_markup=markup)

        
        
@bot.message_handler(content_types=['audio', 'document'])
def handle_audio(message):
    markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
        
    btn1 = types.KeyboardButton("Очистить контекст")

    markup.add(btn1)    

    try:
        msg = bot.send_message(message.chat.id, text="Generating answer...")
        bot.last_message_sent = msg.chat.id, msg.message_id
        cur_query_list = []
        if message.caption is not None:
            cur_query_list.append({'type': 'text', 'content': message.caption})
        else:
            cur_query_list.append({'type': 'text', 'content': 'Describe what you hear in this audio?'})
        filename = str(uuid.uuid4())
        file_name_full="./audio/"+filename
        file_name_full_converted="./ready/"+filename+".wav"
        if message.content_type == 'audio':
            file_info = bot.get_file(message.audio.file_id)
        else:
            file_info = bot.get_file(message.document.file_id)

        downloaded_file = bot.download_file(file_info.file_path)
        with open(file_name_full, 'wb') as new_file:
            new_file.write(downloaded_file)
        os.system("ffmpeg -hide_banner -loglevel error -i "+file_name_full+"  "+file_name_full_converted)
        cur_query_list.append({'type': 'audio', 'content': file_name_full_converted})
        global history_list
        answer, new_history_list = generate_text(model, tokenizer, cur_query_list=cur_query_list, history_list=history_list)
        history_list = new_history_list
        bot.delete_message(*bot.last_message_sent)
        bot.send_message(message.chat.id, text=answer, reply_markup=markup)
    except Exception:
        bot.send_message(message.chat.id, text="Возникла проблема с вашим файлом, убедитесь, что это аудио", reply_markup=markup)


        
    
    
@bot.message_handler(content_types=['voice'])
def handle_voice(message):
    markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
        
    btn1 = types.KeyboardButton("Очистить контекст")

    markup.add(btn1)

    try:
        msg = bot.send_message(message.chat.id, text="Generating answer...")
        bot.last_message_sent = msg.chat.id, msg.message_id
        cur_query_list = []

        filename = str(uuid.uuid4())
        file_name_full="./voice/"+filename+".ogg"
        file_name_full_converted="./ready/"+filename+".wav"
        file_info = bot.get_file(message.voice.file_id)
        downloaded_file = bot.download_file(file_info.file_path)

        with open(file_name_full, 'wb') as new_file:
            new_file.write(downloaded_file)
        os.system("ffmpeg -hide_banner -loglevel error -i "+file_name_full+"  "+file_name_full_converted)
        
        cur_query_list.append({'type': 'text', 'content': 'Describe this voice message?'})
        
        cur_query_list.append({'type': 'audio', 'content': file_name_full_converted})

        global history_list
        answer, new_history_list = generate_text(model, tokenizer, cur_query_list=cur_query_list, history_list=history_list)
        history_list = new_history_list
        bot.delete_message(*bot.last_message_sent)
        bot.send_message(message.chat.id, text=answer, reply_markup=markup)
    except Exception:
        bot.send_message(message.chat.id, text="Возникла проблема с вашим голосовым, попробуйте ещё раз", reply_markup=markup)
        

    
bot.polling(none_stop=True)

In [72]:
cur_query_list

[{'type': 'text', 'content': "Do you know where I've been recently?"},
 {'type': 'image', 'content': 'photo.jpg'},
 {'type': 'audio',
  'content': './ready/9e8aeb40-d2c9-4871-a851-e0c9bb632432.wav'}]

In [13]:
new_history_list

('<s>[INST] You are a useful and friendly interlocutor with great erudition and developed intelligence. You can keep up a conversation on various topics and even know how to play complex intellectual games. What kind of animal is this? Please imagine that you have just heard a sound that probably corresponds to the following text description. "I Love Little Pussy", alternatively called "I Love Little Kitty", is an English language nursery rhyme about a person who is kind to a pet cat. It has a Roud Folk Song Index number of 12824. [/INST]',
 'Based on the description you provided, the animal is most likely a cat. The Roud Folk Song Index number you mentioned corresponds to the song "I Love Little Pussy," which is a nursery rhyme about a person who is fond of their cat. This song is a well-known example of a children\'s game song, which involves the speaker inviting others to play along by repeating the final line, "Here I am again." The language and melody used in the song are typical 

In [17]:
cur_query_list

[{'type': 'text', 'content': 'What is it on the picture?'},
 {'type': 'image', 'content': '6.jpg'}]

### Test

In [8]:
history_list = ("", "")
cur_query_list = [{"type": "text", "content": "Hear the sound? Tell me, what it is?"}, {"type": "audio", "content": "00002.wav"}]
answer, new_history_list = generate_text(model, tokenizer, cur_query_list=cur_query_list, history_list=history_list)
answer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

"I believe that sound is most likely that of a mangrove robin bird. The mangrove robin is a passerine bird species belonging to the family Petroicidae. They are found primarily in mangrove forests in northern Australia, as well as in the Aru Islands of New Guinea. The bird's distinctive call can sound similar to a rustling of leaves, which is why it may be easy to mistake it for other types of sounds."